# Carga de datos a PubSub

Este notebook se utilizará como una máquina que está cargando datos de forma continua a PubSub.
Para conseguir esos datos vamos a coger los datos que se han generado al inicio del curso (datos a nivel día, no datos históricos) y lo vamos a copiarlos en esta carpeta:

```
prevision-dia-tiempo-real
    4. Subiendo la App a Heroku
        Subir datos a PubSub
            TXT_Simulación_TICKETS_YYYY-MM-DD.txt
```


Empecemos cargando los datos

 >- **Nota**: Los datos a introducir tienen que ser del día actual para que el algoritmo funcione. Recordar que esos datos los obtenemos en el paso
 `0. Pasos previos/1. Generando datos/Scripts R/2 - Creación de Tickets tiempo real`

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import json


# TODO: Copia el nombre del archivo copiado en esta carpeta
nombre_archivo = ''

df = pd.read_csv(nombre_archivo, parse_dates=['ticketDate'])

Para continuar, vamos a introducir las credenciales en esta carpeta:

```
C:\path_to_repository\prevision-dia-tiempo-real\4. Subiendo la App a Heroku\Subir datos a PubSub /
    credentials.json
    Carga de datos a PubSub.ipynb
```

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=credentials.json

Ahora, para lanzar nuestros datos a PubSub, tendremos que volver a especificar el tema y el proyecto que hemos creado en Google Cloud

In [ ]:
from google.cloud import pubsub_v1

# TODO especificar nuestro 'project_id' y 'topic_name'
project_id = ''
topic_name = ''

# Configure the batch to publish as soon as there is one kilobyte
# of data or one second has passed.
batch_settings = pubsub_v1.types.BatchSettings(
    max_bytes=1024,  # One kilobyte
    max_latency=1,   # One second
)
publisher = pubsub_v1.PublisherClient(batch_settings)
topic_path = publisher.topic_path(project_id, topic_name)

Una vez que hemos especificado los datos del día, las credenciales y los ID correspondientes, toca subir esos datos. Para ese cometido, vamos a hacerlo en 2 partes:

 - Para empezar vamos a cargar los datos hasta este instante (hasta el minuto actual en el que ejecutamos el código)
 - Una vez hemos cargado los datos iniciales, vamos a ir subiendo los datos minuto a minuto (simulando el tiempo real)

Vamos a cargar todos los tickets hasta el minuto actual

 >- **Nota**: Los datos a introducir tienen que ser del día actual para que el algoritmo funcione. Recordar que esos datos los obtenemos en el paso
 `0. Pasos previos/1. Generando datos/Scripts R/2 - Creación de Tickets tiempo real`

In [ ]:
def first_load():
    now = datetime.now()
    now -= timedelta(seconds=now.second)
    now -= timedelta(microseconds=now.microsecond)
    
    df_now = df[df['ticketDate'] <= now]
    for index, row in df_now.iterrows():
        json_values = {}
        json_values['amount'] = row['amount']
        json_values['ticketDate'] = row['ticketDate'].strftime('%Y-%m-%d %H:%M:%S')
        
        # Convertir diccionario en JSON
        json_values = json.dumps(json_values)
        
        # Codificar String
        json_values = json_values.encode('utf-8')
        future = publisher.publish(topic_path, data=json_values)
        
    print('Mensajes Publicados')

para después subirlos minuto a minuto

In [ ]:
def run():
    now = datetime.now()
    now -= timedelta(seconds=now.second)
    now -= timedelta(microseconds=now.microsecond)
    
    df_now = df[df['ticketDate'] == now]
    for index, row in df_now.iterrows():
        json_values = {}
        json_values['amount'] = row['amount']
        json_values['ticketDate'] = row['ticketDate'].strftime('%Y-%m-%d %H:%M:%S')
        
        # Convertir diccionario en JSON
        json_values = json.dumps(json_values)
        
        # Codificar String
        json_values = json_values.encode('utf-8')
        future = publisher.publish(topic_path, data=json_values)
        
    print('Mensajes Publicados')

Para generar un bucle que se repita cada minuto, vamos a utilizar el módulo `APScheduler` de Python

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

sched = BlockingScheduler()
sched.add_job(run, 'interval', minutes=1)

Ejecutemos...

 >- **Nota**: el siguiente bloque va a hacer que Python se meta en un bucle infinito, por lo que si se quiere parar tendremos que darle al botón de `Stop` o reiniciar el `Kernel`

In [ ]:
first_load()
sched.start()